##### Most of the codes below are copied from '01.mnist_softmax.py'

In [1]:
import os
import tensorflow as tf
import numpy as np
import experiments as exp
from tensorflow import keras

In [2]:
# Define simple model. execution starts from the bottom
def basic_mnist_model():

    model = keras.Sequential([
        keras.layers.Flatten(input_shape=(28, 28)),
        keras.layers.BatchNormalization(),
        keras.layers.Dense(256, activation=tf.nn.relu),

        keras.layers.BatchNormalization(),
        keras.layers.Dense(256, activation=tf.nn.relu),

        keras.layers.BatchNormalization(),
        keras.layers.Dense(256, activation=tf.nn.relu),

        keras.layers.BatchNormalization(),
        keras.layers.Dense(10, activation=tf.nn.softmax)
    ])

    return model

In [3]:
# Load MNIST, FMNIST dataset
mnist = keras.datasets.mnist
(mnist_train_x, mnist_train_y), (mnist_test_x, mnist_test_y)\
    = mnist.load_data()
mnist_train_x, mnist_test_x = mnist_train_x/255., mnist_test_x/255.

fashion_mnist = keras.datasets.fashion_mnist
(fmnist_train_x, fmnist_train_y), (fmnist_test_x, fmnist_test_y)\
    = fashion_mnist.load_data()
fmnist_train_x, fmnist_test_x = fmnist_train_x/255., fmnist_test_x/255.

In [4]:
# Train model if no pre-trained model exists
saved_model_path = './mnist.hdf5'
if not os.path.exists(saved_model_path):

    model = basic_mnist_model()

    # TRAIN MODEL
    training_epochs = 10
    learning_rate = 0.001
    batch_size = 128

    model.compile(optimizer=keras.optimizers.Adam(lr=learning_rate),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    model.fit(mnist_train_x, mnist_train_y,
              epochs=training_epochs,
              batch_size=batch_size)

    test_loss, test_acc = model.evaluate(mnist_test_x, mnist_test_y)
    print("Training done, test accuracy: {}".format(test_acc))

    # SAVE MODEL
    keras.models.save_model(model, "./mnist.hdf5")

else:
    model = keras.models.load_model(saved_model_path)

In [5]:
# right/wrong distinction, regard wrong examples as negative
exp.right_wrong_distinction(model, mnist_test_x, mnist_test_y)

# In-Out-of-distribution test, assumes out-of-distribution samples as negative
exp.in_out_distinction(model,
                       mnist_test_x,
                       fmnist_test_x,
                       "FashionMNIST")

exp.in_out_distinction(model,
                       mnist_test_x,
                       np.random.normal(size=(10000, 28, 28)),
                       "WhiteNoise")

exp.in_out_distinction(model,
                       mnist_test_x,
                       np.random.uniform(size=(10000, 28, 28)),
                       "UniformNoise")



[MNIST SUCCESS DETECTION]
MNIST Error (%)| Prediction Prob (mean, std) | PProb Right                (mean, std) | PProb Wrong (mean, std):
2.2099971771240234 | 0.9896895 0.054563407 | 0.9939357 0.03863608 | 0.8018002 0.18044601
Success base rate (%): 97.79 (9779/10000)
KL[p||u]: Right/Wrong classification distinction
AUPR (%): 99.92
AUROC (%): 96.63
Prediction Prob: Right/Wrong classification distinction
AUPR (%): 99.92
AUROC (%): 96.59

Error Detection
Error base rate (%): 2.21 (221/10000)
AUPR (%): 45.15
AUROC (%): 96.63
Prediction Prob: Right/Wrong classification distinction
AUPR (%): 44.32
AUROC (%): 96.59

[MNIST-FashionMNIST anomaly detection]
In-dist max softmax distribution (mean, std):
0.9896895 0.054563407
Out-of-dist max softmax distribution(mean, std):
0.8990982 0.16524862

Normality Detection
Normality base rate (%): 50.0
KL[p||u]: Normality Detection
AUPR (%): 65.82
AUROC (%): 76.32
Prediction Prob: Normality Detection
AUPR (%): 68.06
AUROC (%): 76.8

Abnormality Detecti